#  Dokumentacja projektu Sentiment Classification
#### Aleksander Ogonowski
   
#### Paweł Rybak

#### Kornel Szymczyk

##### 10 stycznia 2019 

## 1. Instrukcja użytkownika

Pakiet pszt zawiera dwa moduły:
- refactor_csv.py zawierający funkcje pomocnicze do refaktoryzacji zbioru danych Tweetów dotyczących produktów Apple,
- net.py zawierający klasę MLP, która jest definicją modelu sieci neuronowej.

### 1.1. Wymagania uruchomienia
- Python 3.6+
- Numpy

### 1.2. Przekształcenie zbioru danych wykorzystując moduł refactor_csv.py

Przed przystąpieniem do trenowania sieci neuronowej należy przekrzstałcić odpowiednio zbiór danych i zapisać go w nowym pliku csv. W tym celu korzystamy z funkcji *refactor(csv_path, new_csv_path)*. Pierwszy argument funkcji to ścieżka do pliku ze zbiorem danych w formacie *.csv*, drugi argument to ścieżka pliku, w którym zostanie zapisany przekształcony zbiór. 

Przykład kodu:

In [ ]:
from pszt import refactor_csv

# File path to the csv file.
csv_path = 'Apple-Twitter-Sentiment-DFE.csv'

# File path to the new csv file.
new_csv_path = 'apple-twitter_example.csv'
    
refactor_csv.refactor(csv_path, new_csv_path)

Posiadając przekształcony zbiór plik csv należy przystąpić do wektoryzacji zbioru, w tym celu należy skorzystać z funkcji *vectorize_dataset(csv_path, x_train_path, y_train_path, ignore_words=None)*. Jej celem jest wykonanie przekształcenia zdań do modelu bag-of-words oraz zamiana sentymentu zdań (1, 3, 5) do postaci one-hot. Przekształcone zdania zapisywane są w tablicy numpy *x_train*, etykiety w tablicy numpy *y_train* oraz zapisywane są na dysku w formacie *.npy*. Pierwszy argument funkcji to ścieżka do pliku csv utworzonego przy pomocy funkcji *refactor()*, drugi argument to ścieżka pliku do zapisania tablicy *x_train*, trzeci argument to ścieżka pliku do zapisania tablicy *y_train* oraz ostatni argument przyjmuje opcjonalną tablicę słów, które mają być pomijane w zdaniach.

Przykład kodu:

In [ ]:
refactor_csv.vectorize_dataset(new_csv_path, 'x_train_no_ignore_no_norm', 'y_train_no_ignore_no_norm')

### 1.3. Trenowanie sieci neuronowej wykorzystując moduł net.py

W celu wytrenowania sieci tworzymy obiekt klasy MLP. Następnie dodajemy warstwy sieci wykorzystując metodę *add_layer(input_dim, output_dim, activation)*. Pierwszy argument określa liczbę wejść do warstwy, drugi argument określa liczbę wyjść z warstwy, a ostatni określa rodzaj funkcji aktywacji ('relu', 'sigmoid', 'softmax'). Ze względu na to, że etykiety zbioru reprezentowane są w postaci one-hot, ostatnia warstwa powinna zawsze posiadać output_dim=3 oraz activation='softmax'. Model sieci został napisany w taki sposób, że należy pominąć dodawanie warstwy wejściowej sieci neurnowej, a input_dim pierwszej warstwy powinien odpowiadać długości wektora danych wejściowych. Po dodaniu warstw należy je zainicjalizować funkcją *init_layers()*.

Proces trenowania wywołuje się funkcją *train(x, y_true, epochs, silent=False)*. Pierwszy argument to tablica numpy ze zdaniami w postaci bag-of-words, drugi argument to tablicy etykiet zdań w postaci one-hot, trzeci argument określa liczbę kroków trenowania, czwarty agrument opcjonalny określający czy wypisywać na ekran aktualny status trenowania sieci.

Przykład kodu:

In [ ]:
import numpy as np
from pszt import net

# Create a neural net.
mlp = net.MLP();

# Load dataset.
x_train = np.load('x_train_3k.npy')
y_train = np.load('y_train_3k.npy')

# Add layers.
mlp.add_layer(input_dim=x_train.shape[1], output_dim=20, activation="sigmoid")
mlp.add_layer(input_dim=20, output_dim=3, activation="softmax")

# Initialize weights in layers.
mlp.init_layers()

# Train the neural net.
mlp.train(x=x_train, y_true=y_train, epochs=5)

Klasa MLP zawiera również metodę *k_fold_validation(x, y_true, k, epochs)*, która poddaje sieć neuronową walidacji k-fold.

Przykład kodu:

In [ ]:
mlp.k_fold_validation(x=x_train, y_true=y_train, k=5, epochs=5)

## 2. Struktura programu

Najważniejszym elementem programu jest klasa MLP reprezentująca sieć neuronową. Ogólna idea schematu klasy MLP i przechowywania obliczeń sieci została oparta na podstawie artykułu ["Let’s code a Neural Network in plain NumPy"](https://towardsdatascience.com/lets-code-a-neural-network-in-plain-numpy-ae7e74410795). Klasa ta tworzy pustą sieć. Należy do niej dodać warstwy przy pomocy metody *add_layer*. Dodaje ona jedynie informacje o nowej warstwie do metadanych sieci. Aby sieć faktycznie wykorzystała dodaną warstwę należy wywołać metodę *init_layers*. Dla każdej warstwy opisanej w strukturze sieci utworzy ona odpowiadającą macierz wag i przesunięć. Wartości dla wag warstw ukrytych są losowane z rozkładu jednostajnego. Przedstawia to poniższy fragment kodu:

In [ ]:
self.param_values['w' + str(i)] = np.random.uniform(
    low=(-1/np.sqrt(input_size)),
    high=(1/np.sqrt(input_size)),
    size=(input_size, output_size)
)
self.param_values['b' + str(i)] = np.random.uniform(
    low=(-1/np.sqrt(input_size)),
    high=(1/np.sqrt(input_size)),
    size=output_size
)

Wagi warstwy wyjściowej inicjowane są wartością 0.

Klasa ta posiada również funkcję *_forward()*, która wylicza wektor wyjściowy funkcji, w zależności od podanego wektora wejściowego. Dla każdej warstwy sieci wylicza wyjście i funkcję aktywacji, która następnie jest podawane jako wejście do kolejnej warstwy. Każda z pośrednich wartości takiego przejścia jest zapisywana w pamięci sieci. Jest to potrzebne do późniejszej wstecznej propagacji w procesie uczenia sieci. Wspomniana wsteczna propagacja została zaimplementowana w funkcji *_backward()*. Funkcja ta wylicza gradient funkcji kosztu względem wag sieci. Wyliczone gradienty są zapisywane wewnątrz sieci, aby potem mogły być użyte wewnątrz funkcji *_update_weights()*. Modyfikuje ona wagi sieci o wartości proporcjonalne do gradientu. Obliczenia wykonywane przez funkcję *_forward()*, *_backward()* i *_update_weights()* zostały napisane na podstawie artykułu ["Creating a Neural Network from Scratch in Python: Multi-class Classification"](https://stackabuse.com/creating-a-neural-network-from-scratch-in-python-multi-class-classification/).

Uczenie sieci można zainicjować przy pomocy metody *train*, której najważniejszy fragment został przytoczony poniżej.

In [ ]:
def train(self, x, y_true, epochs, silent=False):
    for i in range(1, epochs+1):
        # Perform forward propagation over neural network.
        self._forward(x)

        # Perform backward propagation over neural network.
        self._backward(x, y_true)

        # Update weights of neural network.
        self._update_weights()

Parametrami tej funkcji jest wektor wartości wejściowych, oraz odpowiadającym im wartościom wyjściowym. Wymagane jest również podanie liczby epok, definiującej ile iteracji ma wykonać proces uczenia. Jest to jedyny zaimplementowany warunek stopu.
Uczenie odbywa się na całym zbiorze trenującym podanym do funkcji.

## 3. Kluczowe decyzje projektowe.



### 3.1. One-hot Encoding

W zbiorze danych sentyment zdania został przedstawiony w formie trzech etykiet: "1" - negatywny, "3" - neutralny, "5" - pozytywny. Etykiety zbioru danych zostały zamienione do postaci "1 z n" (ang. One-hot Encoding), czyli w postaci wektora o rozmiarze liczby etykiet w zbiorze, gdzie wartość 1 w wierszu oznacza przynależność do danej klasy. Stosując taką reprezentację, warstwa wyjściowa sieci neuronowej musi mieć funkcję aktywacji Softmax. Funkcja Softmax umożliwia nam prezentację obliczeń sieci w postaci prawdopodobieństwa wystąpienia danej etykiety. Funkcja Softmax:

\begin{equation*}
\ S(z) = \frac{e^{z_i}}{\sum_{j}e^{z_j}}  \\\
\end{equation*}
gdzie:  **S(z)** - wektor wartości funkcji Softmax,

**z** - wektor pierwotnych wyników klasyfikacji.

Posiadając etykiety w postaci "1 z n" sieć oblicza funkcję błędu *Cross-Entropy* według wzoru:

\begin{equation*}
\ Q_i(S(z), y_t) = - \sum_{i} y_{t_i} log(S(z)_i)\\
\end{equation*}

gdzie: Qi - funkcja błędu na zdaniu, 

**S(z)** - wektor wartości funkcji Softmax, 

S(z)i - i-ta wartość wektora S(z), 

**z** - wektor klasyfikacji sieci, 

**yt** - wektor prawdziwych etykiet obrazu w postaci „1 z n”,
        


### 3.2. Bag-of-words

W modelu bag-of-words zdania są reprezentowane w postaci multizbioru słów. Multizbiorem nazywamy zbiór, w którym jeden element może występować wiele razy. W praktyce model ten odnosi się do wektora, w którym dany indeks odpowiada jednemu słowu w zbiorze danych, a wartość wektora i-tego indeksu odpowiada ilości wystąpień słowa w zdaniu.

W takim modelu na uczenie sieci neuronowej najbardziej wpływają słowa, które pojawiają się najczęściej w danym zdaniu.

### 3.3. TFIDF

Model TFIDF pozwala nam na utworzenie wektora, w którym najczęściej występujące słowa w całym zbiorze danych mają najmniejszą wartość, zazwyczaj zerową.

W takim modelu na uczenie sieci neuronowej najbardziej wpływają słowa, które pojawiają się najrzadziej w danym zbiorze.

## 4. Rezultaty i analiza

### 4.1. K-krotna walidacja krzyżowa

Na początku przeprowadzno ocenę różnych modeli sieci neuronowych wykorzystując metodę k-krotnej walidacji krzyżowej. Na początku postanowiliśmy wykorzystać tą metodę z użyciem modelu bag-of-words, proces walidacji został udokumentowany w zeszycie *"kfold-bagofwords-batch.ipynb"*. Zostało stworzonych siedem modeli sieci i k-krotna walidacja krzyżowa została przeprowadzona dla zmiennych lr=10e-4, epochs=300, k=5.

Najmniejszy średni błąd równy **630.10** uzyskała sieć o warstwach:
1. Layer - input_dim: 8257, output_dim: 20, activation: sigmoid
2. Layer - input_dim: 20, output_dim: 3, activation: softmax

Możemy również zauważyć, że dla sieci siódmej, która posiada więcej warstw i więcej neuronów proces uczenia zatrzymał się. Każda iteracja nie powodowała zmian błędu, dlatego proces walidacji został przerwany ręcznie.

    Current fold: T0,
    Len of x_train: 3043,
    Len of x_test: 761.
    Epoch: 10, loss: 725.89, accuracy: 56.9%
    Epoch: 20, loss: 726.06, accuracy: 56.9%
    Epoch: 30, loss: 726.06, accuracy: 56.9%
    Epoch: 40, loss: 726.06, accuracy: 56.9%

Zbiór 'x_bagofwords' ma kształt (3804, 8257). Po analizie wartości sieci okazało się, że wartości liczone przez warstwy były tak duże, że wynikiem funkcji aktywacji (sigmoid, softmax) były wartości nieskończone, dlatego następowały zerowe zmiany wartości błędu. 

Aby zaradzić takiej sytuacji postanowiliśmy napisać nową funkcję *train_minibatch()*, która podczas jednej epoki dzieli zbiór na mniejsze części. Dzięki niej sytuacja nieskończonych wartości powinna być mniej prawdopodobna, ponieważ sieć będzie wykonywała obliczenia na mniejszych wartościach.


Kolejnym krokiem było przeprowadzenie k-krotnej walidacji krzyżowej na zbiorze TFIDF. Proces walidacji został udokumentowany w zeszycie *"kfold-tfidf-batch.ipynb"*. Zostało stworzonych dokładnie takich samych siedem modeli sieci i k-krotna walidacja krzyżowa została przeprowadzona dla zmiennych lr=10e-4, epochs=300, k=5.

Najmniejszy średni błąd równy **714.12** uzyskała sieć o warstwach:
1. Layer - input_dim: 8257, output_dim: 20, activation: relu
2. Layer - input_dim: 20, output_dim: 3, activation: softmax

Również wystąpiła ta sama sytuacja w trenowaniu dla siódmej sieci co w modelu bagofwords.


Możemy zauważyć, że proces uczenia zachodził zdecydowanie szybciej dla modelu bag-of-words niż dla TFIDF. Może być to spodowowane różna interpretacją zdań tych modeli. Może być to również spowodowane różnymi sumarycznymi wartościami dla obu modeli
    np.sum(x_bagofwords) = 66318.0
    np.sum(x_tfidf) = 14093.15.
Mniejsze wartości odpowiadają mniejszym zmianom w wagach sieci podczas uczenia.

#### 4.1.1. Minibatch

Postanowiliśmy ponownie przeprowadzić k-krotna walidację krzyżową dla tych samych modeli wykorzystując metodę dzielienia zbioru trenującego na mniejsze części (minibatch). Udokumentowano to w zeszytach *"kfold-bagofwords-minibatch.ipynb"* oraz *"kfold-tfidf-minibatch.ipynb"*. 

Cały proces wykonania 300 epok trwał zdecydowanie dłużej w przypadku minibatch. Problem nieskończonych wartości zniknął i sieć wyuczyła się do końca w przypadku bagofwords. Ze względu na długi okres trenowania proces uczenia siódmej sieci TFIDF został przerwany ręcznie.

Sieci również osiągnęły lepszy wynik niż w przypadku trenowania na całym zbiorze. W przypadku bag-of-words najmniejszy błąd równy **536.89** osiągnął ten sam model sieci neuronowej. W przypadku TFIDF najmniejszy błąd równy **575.27** osiągneła sieć o warstwach:
1. Layer - input_dim: 8257, output_dim: 50, activation: relu
2. Layer - input_dim: 50, output_dim: 3, activation: softmax.

### 4.2. Trenowanie wybranego modelu sieci neuronowej

Wybrany model wybrany na podstawie dwóch najmniejszych błędów:
1. Layer - input_dim: 8257, output_dim: 20, activation: sigmoid
2. Layer - input_dim: 20, output_dim: 3, activation: softmax.

Po wybraniu modelu sieci neuronowej przeprowadzono proces uczenia, który udokumentowano w zeszycie *"training.ipynb"*. Zbiory danych zostały losowo uporządkowane i 80% zbioru pierwotnego przeznaczono na zbiór trenowania, a 20% zbioru pierwotnego przeznaczono na zbiór testowania. Trenowanie odbyło się korzystając z funkcji *train_batch* o parametrach *lr=10e-4* i *batch_size=256*. Wyniki trenowania sieci przedstawiono na wykresach dokładności na zbiorze trenowania i testowania od epok oraz na wykresach błędu na zbiorze trenowania i testowania od epok.

---

**Dokładności trenowania na bag-of-words:**
![Accuracies 1](images/accuracy_1.png "Accuracies 1")

**Błędy na bag-of-words:**
![Loss 1](images/loss_1.png "Title")

    Wyniki po ostatniej epoce:
    loss_train: 518.67, accuracy_train: 95.4%
    loss_test: 503.57, accuracy_test: 76.0%
    
Możemy zauważyć, że przez cały proces uczenia błąd na zbiorze trenowanującym ciągle malał, jednak na zbiorze testującym już nie. W 170 epoce strata na zbiorze testującym zaczęła rosnąć. Gdybyśmy zaczęli trenować sieć dłużej zapewne wystąpiłoby zjawisko przeuczenia, które charakteryzuje się większym błędem na zbiorze testowania niż na zbiorze trenowania.

---

---
**Dokładności trenowania na TFIDF:**
![Accuracies 2](images/accuracy_2.png "Accuracies 2")

**Błędy na TFIDF:**
![Loss 2](images/loss_2.png "Loss 2")

    Wyniki po ostatniej epoce:
    loss_train: 1819.03, accuracy_train: 80.1%
	loss_test: 541.29, accuracy_test: 73.7%

Sieć uzyskała zdecydowanie gorsze wyniki w porównaniu do modelu bag-of-words. Jednak możemy zauważyć, że wszystkie wartości stale się ulepszały. Dlatego postanowiono wytrenować tą samą sieć od początku jednak wykonując 1200 epok, co przedstawiono poniżej oraz w zeszycie *training-2.ipynb*.

---

---
**Dokładności trenowania na TFIDF:**
![Accuracies 3](images/accuracy_3.png "Accuracies 3")

**Błędy na TFIDF:**
![Loss 3](images/loss_3.png "Loss 3")

    Wyniki po ostatniej epoce:
    loss_train: 324.15, accuracy_train: 96.5%
	loss_test: 531.03, accuracy_test: 77.1%
    
Możemy zauważyć, że w okolicach 900 epoki wystąpiło zjawisko przeuczenia. Osiągnięto również lepsze wyniki w porównaniu do bag-of-words oprócz straty na zbiorze testowania.
    
---

## 4.3. Wnioski

- Trenowanie na zbiorze wykorzystujacym model bag-of-words osiągnęło podobne rezultaty co model TFIDF wykonując 3 krotnie mniej epok. Jest to zdecydowana oszczędność czasu jak i kosztów. Jednak wykorzystując model TFIDF możliwe jest osiągnięcie lepszych rezultatów większym nakładem czasu i kosztów. 

- Obie sieci osiągnęły dokładność na zbiorze trenowania na poziomie 96% i na zbiorze testowania w okolicach 76%. Oznacza to, że wytrenowa sieć w miarę dobrze rozpoznaje składowe wpływające na dany sentyment w zbiorze trenowania jednak nie nauczyła się efektywnie generalizować innych zbiorów danych.

- W celu zapobiegania zjawiska przeuczenia należałoby zastosować metodę wczesnego zatrzymania uczenia, bądź dropout i ponownie wykonać trenowanie sieci.

- W zbiorze uwzględniono wszystkie słowa zawierające znaki alfanumeryczne. Możliwe, że niektóre powtarzające się słowa nie mają żadnego wpływu na sentyment i stanowią pewien szum podczas trenowania, dlatego należałoby przefiltrować zbiór danych dokładniej i ponownie wytrenować sieć.

- W celu próby usprawnienia wyników sieci neuronowej można również spróbować dodać ze zbioru danych do wektora zdań dzień napisania tweeta. 

- W celu dalszej analizy prawdopodobnie dobrym pomysłem by było wykonanie analizy w stylu Heat Map polegające na iteracyjnym zastąpieniu każdego elementu wektora zdania zerową wartością i rejestracją klasyfikacji. W przypadku gdy zdanie jest poprawnie klasyfikowane z dużym prawdopodobieństwem taka analiza mogłaby nam pokazać, który element wpływa na poprawną klasyfikację. W odwrotnym przypadku gdy zdanie jest błędnie klasyfikowane mołoby nam pokazać, który element zdania błędnie wpływa na poprawną klasyfikację.

- W celu dalszej analizy należałoby wykonać tablicę pomyłek (ang. confusion matrix) i zweryfikować, która klasa jest klasyfikowana niepoprawnie i spróbować wyciągnąć dalsze wnioski z tego.